Jacob Graham

Hydrogen Bond Count

In [24]:
import MDAnalysis as mda
from MDAnalysis.analysis.distances import distance_array
import os
import pickle as pkl
import numpy as np

In [25]:
out_path = 'Count_HBonds_out'
if os.path.exists(out_path)==0:
    os.mkdir(out_path)

In [28]:
cutoff_distance = 1.0
rand_seed = 11111
dcd_list = [f'stretch_{rand_seed}.dcd']
psf_file = "spider_HA1B1A1B1x2134_silkworm_A1B1x0_protein_only_for_visualization.psf"
# Avoid rerunning analysis if data has already been saved in pickled format.
if os.path.exists(f'{out_path}/type1_contacts_exclude_bonded_atoms.pkl') == True:
    total_type1_contacts, inter_chain_type1_contacts, ratio_total_inter_chain_contacts = pkl.load(open(f"{out_path}/type1_contacts_exclude_bonded_atoms.pkl", "rb"))
else:
    # Load dcds into mda universe.
    u = mda.Universe(psf_file, dcd_list)
    # Make an atom group containing only the type 1 crystalline atoms.
    u_all_crystal = u.select_atoms('name 1')
    # Calculate all distances between atoms in group u_all_crystal
    box = u.dimensions
    distances = distance_array(u_all_crystal, u_all_crystal, box=box)
    # Get the indices for all distances that are between 0 and the cutoff distance. This will automatically exclude self distance values which are automatically 0.0.
    indices = np.argwhere((distances > 0) & (distances <= cutoff_distance))
    # Exclude duplicate index pairs, which were computed by distance_array function.
    lst = set([tuple(sorted(i)) for i in indices])
    lst = list(map(list, lst))
    # Calculate total contacts, inter chain contacts, and ratio of the two values.
    total_type1_contacts = 0
    inter_chain_type1_contacts = 0
    for i, j in lst:
        if abs(u_all_crystal[i].index - u_all_crystal[j].index) > 1:
            total_type1_contacts +=1
            if u_all_crystal[i].resid != u_all_crystal[j].resid:
                inter_chain_type1_contacts+=1
    # Calculate ratio of interchain to total type 1 contacts.
    ratio_total_inter_chain_contacts = inter_chain_type1_contacts/total_type1_contacts
    # Pickle data.
    saveObject = (total_type1_contacts, inter_chain_type1_contacts, ratio_total_inter_chain_contacts)
    pkl.dump(saveObject, open(f"{out_path}/type1_contacts_exclude_bonded_atoms.pkl", 'wb'))
print(f'Total Hydrogen Bonds = {total_type1_contacts}')
print(f'Total Inter Chain Hydrogen Bonds = {inter_chain_type1_contacts}')
print(f'Ratio of Inter Chain/Total Hydrogen Bonds = {ratio_total_inter_chain_contacts}')

Total Hydrogen Bonds = 98236
Total Inter Chain Hydrogen Bonds = 91168
Ratio of Inter Chain/Total Hydrogen Bonds = 0.9280508164013193
